In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI流水线: 使用google-cloud-pipeline-components进行模型上传、预测和评估

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pipelines/google_cloud_pipeline_components_model_upload_predict_evaluate.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pipelines/google_cloud_pipeline_components_model_upload_predict_evaluate.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/pipelines/google_cloud_pipeline_components_model_upload_predict_evaluate.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
     </a>
  </td>
</table>
<br/><br/><br/>
*注意: 本笔记本使用KFP 1.x和GCPC 1.x。我们建议使用2.x*

## 概述

本笔记本展示如何结合实验性的`evaluation`方法，使用[`google_cloud_pipeline_components`](https://github.com/kubeflow/pipelines/tree/master/components/google-cloud)中定义的组件，构建一个[Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines)工作流程。该工作流程将上传一个表格自定义模型作为`Model`资源，创建一个`BatchPredictionJob`资源，然后评估`Model`资源和`BatchPredictionJob`结果，生成一个评估`system.Metrics`工件。

了解更多关于[Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction)和[Vertex AI Model components](https://cloud.google.com/vertex-ai/docs/pipelines/model-endpoint-component)的信息。

### 目标

在本教程中，您将学习如何使用来自`google_cloud_pipeline_components`的组件和您构建的自定义管道组件来评估自定义模型。

本教程使用以下Google Cloud ML服务和资源：

- Vertex AI管道
- Vertex AI模型注册表
- Vertex AI批量预测

执行的步骤包括：

- 将预训练模型上传为`Model`资源。
- 在`Model`资源上运行`BatchPredictionJob`并使用真实数据。
- 生成关于`Model`资源的评估`Metrics`工件。
- 将评估指标与阈值进行比较。

数据集

本教程使用的数据集是[安全驾驶员预测Kaggle竞赛](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction/overview)的一部分。模型已经在这些数据上进行了训练，并使用了地面真相进行评估。

该数据集预测了保单持有人是否提交了索赔。

费用

本教程使用 Google Cloud 的计费组件：

- Vertex AI
- Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用量生成费用估算。

安装以下包以执行此笔记本所需的操作。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 'google-cloud-pipeline-components<2' \
                                 'kfp<2'

只有协作人员：取消注释下面的单元格以重新启动内核。

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 在开始之前

### 设置您的Google云项目

**无论您使用的是笔记本环境都需要执行以下步骤。**

1. [选择或创建一个Google云项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您将获得$300的免费信用额用于您的计算/存储费用。

2. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用Vertex AI API]

4. 如果您在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

#### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

UUID（Universally Unique Identifier）

如果您正在进行实时教程，您可能正在使用共享的测试帐户或项目。为了避免在创建的资源上产生名称冲突，您需要为每个实例会话创建一个uuid，并将其附加到您在本教程中创建的资源的名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 对您的Google Cloud账户进行身份验证

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照以下相关说明操作。

**1. Vertex AI Workbench**
* 无需操作，因为您已经过身份验证。

**2. 本地JupyterLab实例，取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 协作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务账户或其他
* 请参考如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples上向您的服务账户授予Cloud Storage权限。

创建一个云存储桶

创建一个存储桶来存储诸如数据集之类的中间工件。

- *{给笔记本作者的注：对于任何需要是唯一的用户提供的字符串（如存储桶名称或模型ID），请在末尾添加“-unique”，以便进行适当的测试}*

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶尚不存在时才能运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

### 服务账号

您可以使用服务账号来创建 Vertex AI Pipeline 作业。

如果您不想使用项目的 Compute Engine 服务账号，请将`SERVICE_ACCOUNT`设置为另一个服务账号的ID。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

为 Vertex AI 流水线设置服务帐户访问权限

运行以下命令以授予您的服务帐户访问权限，可以在您之前创建的存储桶中读取和写入流水线工件 — 您只需要对每个服务帐户运行一次这些命令。

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

导入库并定义常量

In [ ]:
import kfp
from google.cloud import aiplatform
from google_cloud_pipeline_components.experimental.evaluation import \
    ModelEvaluationOp as evaluation_op
from google_cloud_pipeline_components.types import artifact_types
from google_cloud_pipeline_components.v1.batch_predict_job import \
    ModelBatchPredictOp as batch_prediction_op
from google_cloud_pipeline_components.v1.model import \
    ModelUploadOp as model_upload_op
from kfp.v2 import compiler
from kfp.v2.components import importer_node
from kfp.v2.dsl import Input, Metrics, component

为Vertex AI管道设置以下常量：

In [ ]:
PIPELINE_ROOT = "{}/pipeline_root/safe_driver".format(BUCKET_URI)

### 初始化用于Python的Vertex AI SDK

为您的项目和相应的存储桶初始化Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI, location=REGION)

创建组件用于将评估指标与阈值进行比较

首先，您可以创建自己的组件，该组件以评估指标文档作为输入，检查阈值并返回是/否决定。它将用于随后的dsl.Condition()中，以决定模型是否应继续下一步，即在线部署。

该组件接受以下参数：

- `eval_metrics`：从`ModelEvaluation`组件返回的评估指标文档。
- `metric_name`：指标条目的键名，用于进行比较。
- `threshold`：用于进行是/否决定的指标值阈值。

In [ ]:
@component()
def compare(eval_metrics: Input[Metrics], metric_name: str, threshold: float) -> str:
    path = eval_metrics.path
    # print("PATH", path)

    gs_prefix = "gs://"
    gcsfuse_prefix = "/gcs/"
    if path.startswith(gs_prefix):
        path = path.replace(gs_prefix, gcsfuse_prefix)

    import json

    with open(path, "r") as f:
        data = json.load(f)

    slices = data["slicedMetrics"]

    metrics = slices[0]["metrics"]["classification"]
    # print("METRIC KEYS", metrics.keys())

    value = metrics[metric_name]
    if value > threshold:
        return "true"

    return "false"

##定义使用`google_cloud_pipeline_components`组件的评估流水线

接下来，定义流水线。

用于定义流水线的[`google_cloud_pipeline_components`](https://github.com/kubeflow/pipelines/tree/master/components/google-cloud)组件包括：上传模型、运行批量预测以及使用给定的预测评估模型。

查看[上传模型组件](https://github.com/kubeflow/pipelines/blob/master/components/google-cloud/google_cloud_pipeline_components/aiplatform/model/upload_model/component.yaml)的定义。

查看[批量预测组件](https://github.com/kubeflow/pipelines/blob/master/components/google-cloud/google_cloud_pipeline_components/aiplatform/batch_predict_job/component.yaml)的定义。

查看[实验性评估组件](https://github.com/kubeflow/pipelines/blob/master/components/google-cloud/google_cloud_pipeline_components/experimental/evaluation/component.yaml)的定义。

In [ ]:
DATA_URIS = [
    "gs://cloud-samples-data/vertex-ai/dataset-management/datasets/safe_driver/dataset_safe_driver_train_10k.csv"
]
MODEL_URI = "gs://cloud-samples-data/vertex-ai/google-cloud-aiplatform-ci-artifacts/models/safe_driver/model"
# Create working dir
WORKING_DIR = f"{PIPELINE_ROOT}/{UUID}"
MODEL_DISPLAY_NAME = f"safe-driver-{UUID}"
BATCH_PREDICTION_DISPLAY_NAME = f"batch-prediction-on-pipelines-model-{UUID}"


@kfp.dsl.pipeline(name="upload-evaluate-" + UUID)
def pipeline(
    metric: str,
    threshold: float,
    project: str = PROJECT_ID,
    model_display_name: str = MODEL_DISPLAY_NAME,
    batch_prediction_display_name: str = BATCH_PREDICTION_DISPLAY_NAME,
    batch_prediction_data_uris: list = DATA_URIS,
):
    import_unmanaged_model_task = importer_node.importer(
        artifact_uri=MODEL_URI,
        artifact_class=artifact_types.UnmanagedContainerModel,
        metadata={
            "artifactUri": MODEL_URI,
            "predictSchemata": {
                "predictionSchemaUri": MODEL_URI + "/prediction_schema.yaml",
                "instanceSchemaUri": MODEL_URI + "/instance.yaml",
            },
            "containerSpec": {
                "imageUri": "us-docker.pkg.dev/vertex-ai/automl-tabular/prediction-server:prod",
                "healthRoute": "/health",
                "predictRoute": "/predict",
            },
        },
    )

    model_task = model_upload_op(
        project=project,
        display_name=model_display_name,
        unmanaged_container_model=import_unmanaged_model_task.outputs["artifact"],
    )

    batch_prediction_task = batch_prediction_op(
        project=project,
        model=model_task.outputs["model"],
        job_display_name=batch_prediction_display_name,
        gcs_source_uris=batch_prediction_data_uris,
        instances_format="csv",
        predictions_format="jsonl",
        gcs_destination_output_uri_prefix=WORKING_DIR,
        machine_type="n1-standard-32",
    )

    eval_task = evaluation_op(
        project=project,
        root_dir=WORKING_DIR,
        problem_type="classification",
        classification_type="multiclass",
        ground_truth_column="target",
        class_names=["0", "1"],
        predictions_format="jsonl",
        batch_prediction_job=batch_prediction_task.outputs["batchpredictionjob"],
    )

    _ = compare(
        eval_metrics=eval_task.outputs["evaluation_metrics"],
        metric_name=metric,
        threshold=threshold,
    )

编译管道。

In [ ]:
compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path="evaluation_demo_pipeline.json",
)

运行管道

接下来，运行管道。

In [ ]:
DISPLAY_NAME = "safe_driver" + UUID

job = aiplatform.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path="evaluation_demo_pipeline.json",
    pipeline_root=PIPELINE_ROOT,
    parameter_values={"metric": "auPrc", "threshold": 0.95},
    enable_caching=True,
)

job.run()

! rm evaluation_demo_pipeline.json

点击生成的链接，查看在云控制台中的运行情况。

在用户界面中，当您点击节点时，管道DAG的节点会展开或折叠。

查看管道结果和评估指标

In [ ]:
PROJECT_NUMBER = job.gca_resource.name.split("/")[1]
print(PROJECT_NUMBER)


def print_pipeline_output(job, output_task_name):
    JOB_ID = job.name
    print(JOB_ID)
    artifact = ""
    for _ in range(len(job.gca_resource.job_detail.task_details)):
        TASK_ID = job.gca_resource.job_detail.task_details[_].task_id
        EXECUTE_OUTPUT = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/executor_output.json"
        )
        GCP_RESOURCES = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/gcp_resources"
        )
        EVALUATION_METRICS = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/evaluation_metrics"
        )
        # Check if file exists, 0 is success
        !gsutil -q stat $EXECUTE_OUTPUT
        if _exit_code == 0:
            ! gsutil cat $EXECUTE_OUTPUT
            artifact = EXECUTE_OUTPUT
            break
        !gsutil -q stat $GCP_RESOURCES
        if _exit_code == 0:
            ! gsutil cat $GCP_RESOURCES
            artifact = GCP_RESOURCES
            break
        !gsutil -q stat $EVALUATION_METRICS
        if _exit_code == 0:
            ! gsutil cat $EVALUATION_METRICS
            artifact = EVALUATION_METRICS
            break

    return artifact


print("model-upload")
artifacts = print_pipeline_output(job, "model-upload")
print("\n\n")
print("model-batch-predict")
artifacts = print_pipeline_output(job, "model-batch-predict")
print("\n\n")
print("model-evaluation")
metrics = print_pipeline_output(job, "model-evaluation")
print("\n\n")
print("compare")
artifacts = print_pipeline_output(job, "compare")

### 删除管道作业

在管道作业完成后，您可以使用 `delete()` 方法来删除管道作业。在完成之前，可以使用 `cancel()` 方法来取消管道作业。

In [ ]:
job.delete()

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的 [Google Cloud 项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

- 模型
- 批处理作业
- Cloud 存储桶（将 `delete_bucket` 设置为 **True** 以删除 Cloud 存储桶）。

In [ ]:
import os

delete_bucket = False

# Delete the created model
models = aiplatform.Model.list(
    filter=f"display_name={MODEL_DISPLAY_NAME}", order_by="create_time"
)
if len(models) > 0:
    model = models[0]
    model.delete()
    print("Deleted model:", model)

# Delete the created batch-prediction job
batch_predictions = aiplatform.BatchPredictionJob.list(
    filter=f"display_name={BATCH_PREDICTION_DISPLAY_NAME}",
    order_by="create_time",
)
if len(batch_predictions) > 0:
    batch_prediction = batch_predictions[0]
    batch_prediction.delete()
    print("Deleted batch prediction job:", batch_prediction)

# Delete the Cloud Storage bucket
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI